# Elon X Daily Summarizer Lab

## Overview
This lab demonstrates how to build an AI-powered daily brief that summarizes a Twitter/X account's recent activity. We use:
- **twitterapi.io** - Third-party API to fetch tweets
- **OpenAI GPT-4.1-mini** - LLM to analyze and summarize content

## What You'll Learn
1. Fetching tweets from an external API
2. Parsing and formatting data for LLM consumption
3. Crafting effective system/user prompts for summarization
4. Comparing "today vs yesterday" activity patterns

## Prerequisites
- `.env` file with `TWITTERAPI_IO_KEY` and `OPENAI_API_KEY`
- Python packages: `openai`, `requests`, `python-dotenv`

In [1]:
# Imports and Setup
import os
import time
import json
import requests
import datetime as dt
from zoneinfo import ZoneInfo
from email.utils import parsedate_to_datetime

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

# Load environment variables
load_dotenv(override=True)

TWITTERAPI_IO_KEY = os.getenv("TWITTERAPI_IO_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not TWITTERAPI_IO_KEY:
    raise ValueError("Missing TWITTERAPI_IO_KEY in your .env file")
if not OPENAI_API_KEY:
    raise ValueError("Missing OPENAI_API_KEY in your .env file")

openai = OpenAI()

# Configuration
BASE_URL = "https://api.twitterapi.io"
TZ = ZoneInfo("America/Chicago")

print("Setup complete!")

Setup complete!


In [2]:
# Fetch Tweets from twitterapi.io
# Free-tier: 1 request every 5 seconds, includes 429 backoff handling

def fetch_user_last_tweets(username: str, limit: int = 80) -> list[dict]:
    """Fetch recent tweets for a username via twitterapi.io"""
    url = f"{BASE_URL}/twitter/user/last_tweets"
    headers = {"X-API-Key": TWITTERAPI_IO_KEY}
    params = {"userName": username, "count": limit, "limit": limit}

    r = requests.get(url, headers=headers, params=params, timeout=30)

    # Retry once if rate-limited
    if r.status_code == 429:
        time.sleep(6)
        r = requests.get(url, headers=headers, params=params, timeout=30)

    if not r.ok:
        raise RuntimeError(f"TwitterAPI error {r.status_code}: {r.text[:800]}")

    data = r.json()
    
    # Handle various response shapes from the API
    tweets = (
        data.get("data", {}).get("tweets")
        or data.get("tweets")
        or data.get("data", {}).get("items")
        or data.get("data")
        or []
    )

    if isinstance(tweets, dict):
        tweets = tweets.get("tweets") or tweets.get("items") or []

    return tweets[:limit]

In [4]:
# Tweet Parsing Helpers

def _parse_created_at(tweet: dict) -> dt.datetime | None:
    """Parse tweet timestamp and convert to local timezone"""
    s = tweet.get("createdAt") or tweet.get("created_at")
    if not s:
        return None
    try:
        return parsedate_to_datetime(s).astimezone(TZ)
    except Exception:
        return None

def is_retweet(tweet: dict) -> bool:
    return tweet.get("retweeted_tweet") is not None

def tweet_text(tweet: dict) -> str:
    return (tweet.get("text") or "").strip()

def tweet_url(tweet: dict) -> str:
    return (tweet.get("url") or tweet.get("twitterUrl") or "").strip()

def split_today_vs_yesterday(tweets: list[dict], include_retweets: bool = True) -> tuple[list[dict], list[dict]]:
    """Separate tweets into today's and yesterday's buckets"""
    now = dt.datetime.now(TZ)
    today, yesterday = now.date(), (now - dt.timedelta(days=1)).date()
    
    todays, yesterdays = [], []
    for tw in tweets:
        if not include_retweets and is_retweet(tw):
            continue
        t = _parse_created_at(tw)
        if not t:
            continue
        if t.date() == today:
            todays.append(tw)
        elif t.date() == yesterday:
            yesterdays.append(tw)
    
    sort_key = lambda tw: (_parse_created_at(tw) or dt.datetime.min.replace(tzinfo=TZ)).timestamp()
    return sorted(todays, key=sort_key), sorted(yesterdays, key=sort_key)

In [5]:
# Format Tweets for LLM Consumption

def format_tweets_for_llm(tweets: list[dict], max_items: int = 50) -> str:
    """Convert tweet list to a compact text format for the LLM"""
    if not tweets:
        return "(no tweets found)"
    
    lines = []
    for tw in tweets[:max_items]:
        t = _parse_created_at(tw)
        ts = t.strftime("%Y-%m-%d %H:%M %Z") if t else "UNKNOWN_TIME"
        prefix = "RT: " if is_retweet(tw) else ""
        text = tweet_text(tw)
        url = tweet_url(tw)
        lines.append(f"- [{ts}] {prefix}{text} ({url})".strip())
    return "\n".join(lines)

# System Prompt - Defines the AI's role and output format
system_prompt = """
You are a sassy snarky intelligence analyst.
You will be given two sets of tweets from the same account:
(1) today's tweets
(2) yesterday's tweets

Produce a crisp daily brief with these sections:

1) What changed vs yesterday?
2) What's new product/market signal?
3) Anything controversial/risky?
4) Actionable items:
   - Investigate (max 3)
   - Monitor (max 3)
   - Ignore (max 3)

Rules:
- Ground everything in the provided tweets
- If today's tweets are empty, say so and base "changed vs yesterday" on that
- Output Markdown only (no code block)
""".strip()

In [6]:
# Core Summarization Function

def summarize_daily(username: str = "elonmusk", include_retweets: bool = True) -> str:
    """Fetch tweets and generate an AI-powered daily summary"""
    
    # Step 1: Fetch recent tweets
    raw = fetch_user_last_tweets(username, limit=80)
    
    # Step 2: Split into today vs yesterday
    todays, yesterdays = split_today_vs_yesterday(raw, include_retweets=include_retweets)
    
    # Step 3: Format for LLM
    today_block = format_tweets_for_llm(todays, max_items=50)
    yday_block = format_tweets_for_llm(yesterdays, max_items=50)
    
    # Step 4: Build user prompt
    user_prompt = f"""
Account: @{username}
Timezone: America/Chicago
Include retweets: {include_retweets}

TODAY TWEETS:
{today_block}

YESTERDAY TWEETS:
{yday_block}
""".strip()
    
    # Step 5: Call OpenAI
    resp = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
    )
    return resp.choices[0].message.content

def display_summary(username: str = "elonmusk", include_retweets: bool = True):
    """Display the summary as formatted Markdown"""
    md = summarize_daily(username=username, include_retweets=include_retweets)
    display(Markdown(md))

In [7]:
# Run the Daily Summary
# Change username to analyze any Twitter/X account

display_summary("elonmusk", include_retweets=True)

# Daily Brief – @elonmusk – 2026-01-26 (America/Chicago)

## 1) What changed vs yesterday?
- Today’s tweeting activity is significantly lighter, consisting mostly of retweets and a single one-word original tweet ("True"), versus yesterday’s extensive retweet spree mixed with a few original posts including updates about Starship and Cybertruck.
- The tone today is more reflective and nostalgic (e.g., retweet about Elon needing business cards) and continues the theme of Grok AI engagement, whereas yesterday had more technical and promotional content on space, AI, and product missions.
- No direct product launch or technical update today compared to yesterday’s Starship launch timeline announcement ("in 6 weeks").

## 2) What's new product/market signal?
- Continued emphasis on Grok AI's capabilities including watching videos and analyzing content, implying ongoing development or marketing of Grok’s multi-modal intelligence and real-world applications.
- No new explicit product launches or major market signals today; focus remains on Grok and legacy content about SpaceX missions.

## 3) Anything controversial/risky?
- No new controversial or risky statements today; today’s content is primarily retweets and a simple “True” post without inflammatory or polarizing comments.
- The tone is neutral; no escalations or disputes compared to prior mentions of "Rioter signal chat" or sensitive political material found in yesterday’s tweets.

## 4) Actionable items

### Investigate
- Grok AI’s recent functional upgrades hinted by retweets about Grok watching videos and its emotional spectrum analysis capabilities.
- Verify if the “True” tweet today refers to a broader context or forthcoming announcement.
- Social media engagement metrics change due to lighter activity today; check for any strategic shifts.

### Monitor
- Starship launch developments, as the 6-week timeline was announced yesterday—look for updates or delays.
- Grok AI mentions and publicity, including performance in real-world simulations and multimedia analysis.
- Cybertruck operational performance stories and viral content indicating market reception.

### Ignore
- Nostalgic or humorous retweets not adding strategic insight (e.g., business cards reference).
- General music or art posts retweeted (e.g., peaceful music, Grok artistic imagery).
- Broad non-technical philosophical posts about code or consciousness without clear product relevance.

## Next Steps & Customization Ideas

- **Change the target account**: Replace `"elonmusk"` with any public Twitter handle
- **Modify the system prompt**: Adjust the analyst's tone or focus areas
- **Extend the time window**: Modify `split_today_vs_yesterday` to compare longer periods
- **Add sentiment analysis**: Include sentiment scoring in the output
- **Schedule daily runs**: Use cron or a cloud function to automate daily briefs